# Πράκτορες Azure AI με Υποστήριξη Πρωτοκόλλου Πλαισίου Μοντέλου (MCP)

Αυτό το σημειωματάριο δείχνει πώς να χρησιμοποιήσετε τους Πράκτορες Azure AI με εργαλεία Πρωτοκόλλου Πλαισίου Μοντέλου (MCP) για να δημιουργήσετε έναν έξυπνο πράκτορα που μπορεί να αξιοποιήσει εξωτερικούς MCP servers για ενισχυμένες δυνατότητες.


## Εγκατάσταση Απαραίτητων Πακέτων NuGet

Αρχικά, πρέπει να εγκαταστήσουμε το πακέτο Azure AI Agents Persistent, το οποίο παρέχει τη βασική λειτουργικότητα για την εργασία με τους Azure AI Agents.


## Οφέλη Επαλήθευσης Χωρίς Κλειδιά

Αυτό το σημειωματάριο παρουσιάζει την **επαλήθευση χωρίς κλειδιά**, η οποία προσφέρει αρκετά πλεονεκτήματα:
- ✅ **Χωρίς διαχείριση API keys** - Χρησιμοποιεί επαλήθευση βασισμένη στην ταυτότητα του Azure
- ✅ **Ενισχυμένη ασφάλεια** - Δεν αποθηκεύονται μυστικά στον κώδικα ή στη διαμόρφωση
- ✅ **Αυτόματη ανανέωση διαπιστευτηρίων** - Το Azure διαχειρίζεται τον κύκλο ζωής των διαπιστευτηρίων
- ✅ **Πρόσβαση βάσει ρόλων** - Χρησιμοποιεί το Azure RBAC για λεπτομερή δικαιώματα

Το `DefaultAzureCredential` θα χρησιμοποιήσει αυτόματα την καλύτερη διαθέσιμη πηγή διαπιστευτηρίων:
1. Διαχειριζόμενη Ταυτότητα (όταν εκτελείται στο Azure)
2. Διαπιστευτήρια Azure CLI (κατά την ανάπτυξη)
3. Διαπιστευτήρια Visual Studio
4. Μεταβλητές περιβάλλοντος (αν έχουν διαμορφωθεί)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Εγκαταστήστε το πακέτο Azure Identity για έλεγχο ταυτότητας με τις υπηρεσίες Azure χρησιμοποιώντας το DefaultAzureCredential.


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## Εισαγωγή Απαραίτητων Χώρων Ονομάτων

Εισάγετε τους απαραίτητους χώρους ονομάτων για τους Azure AI Agents και το Azure Identity.


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Ρύθμιση του Azure AI Agent Client (Αυθεντικοποίηση χωρίς κλειδί)

Ρυθμίστε τις μεταβλητές παραμέτρων και δημιουργήστε το PersistentAgentsClient χρησιμοποιώντας **αυθεντικοποίηση χωρίς κλειδί**:
- **projectEndpoint**: Το endpoint του έργου Azure AI Foundry
- **modelDeploymentName**: Το όνομα του αναπτυγμένου AI μοντέλου (GPT-4.1 nano)
- **mcpServerUrl**: Η διεύθυνση URL του MCP server (Microsoft Learn API)
- **mcpServerLabel**: Μια ετικέτα για την αναγνώριση του MCP server
- **DefaultAzureCredential**: Χρησιμοποιεί διαχειριζόμενη ταυτότητα, Azure CLI ή άλλες πηγές διαπιστευτηρίων (δεν απαιτούνται API keys)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## Δημιουργία Ορισμού Εργαλείου MCP

Δημιουργήστε έναν ορισμό εργαλείου MCP που συνδέεται με τον διακομιστή MCP του Microsoft Learn. Αυτό θα επιτρέψει στον πράκτορα να έχει πρόσβαση στο περιεχόμενο και την τεκμηρίωση του Microsoft Learn.


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## Δημιουργία του AI Agent

Δημιουργήστε έναν μόνιμο AI agent με το καθορισμένο μοντέλο και τα εργαλεία MCP. Ο agent έχει διαμορφωθεί με:
- Το μοντέλο GPT-4.1 nano
- Οδηγίες για τη χρήση των εργαλείων MCP για βοήθεια
- Πρόσβαση στον διακομιστή Microsoft Learn MCP


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## Δημιουργία Νήματος και Αποστολή Μηνύματος

Δημιουργήστε ένα νήμα συνομιλίας και στείλτε ένα μήνυμα χρήστη ρωτώντας για τη διαφορά μεταξύ του Azure OpenAI και του OpenAI. Αυτό θα δοκιμάσει την ικανότητα του πράκτορα να χρησιμοποιεί τα εργαλεία MCP για να παρέχει ακριβείς πληροφορίες.


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## Ρύθμιση Πόρων Εργαλείου MCP (Χωρίς Κλειδί)

Ρυθμίστε τους πόρους του εργαλείου MCP. Για μια πραγματικά προσέγγιση χωρίς κλειδί, μπορείτε να αφαιρέσετε προσαρμοσμένες κεφαλίδες εάν ο διακομιστής MCP υποστηρίζει έλεγχο ταυτότητας που βασίζεται στην ταυτότητα του Azure. Το παρακάτω παράδειγμα δείχνει πώς να προσθέσετε κεφαλίδες εάν χρειάζεται, αλλά για σενάρια χωρίς κλειδί, αυτές μπορεί να μην είναι απαραίτητες.


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## Έναρξη Εκτέλεσης Πράκτορα

Δημιουργήστε και ξεκινήστε μια εκτέλεση για να επεξεργαστείτε το μήνυμα του χρήστη. Ο πράκτορας θα χρησιμοποιήσει τα διαμορφωμένα εργαλεία και πόρους MCP για να δημιουργήσει μια απάντηση.


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## Παρακολούθηση Εκτέλεσης και Διαχείριση Εγκρίσεων Εργαλείων (Χωρίς Κλειδί)

Παρακολουθήστε την κατάσταση εκτέλεσης του agent και διαχειριστείτε τυχόν απαιτούμενες εγκρίσεις εργαλείων. Αυτός ο βρόχος:
1. Περιμένει να ολοκληρωθεί η εκτέλεση ή να απαιτηθεί ενέργεια
2. Εγκρίνει αυτόματα τις κλήσεις εργαλείων MCP όταν χρειάζεται
3. Για αυθεντικοποίηση χωρίς κλειδί, οι κεφαλίδες μπορεί να μην είναι απαραίτητες εάν ο διακομιστής MCP υποστηρίζει ταυτότητα Azure


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## Εμφάνιση Αποτελεσμάτων Συνομιλίας

Ανάκτηση και εμφάνιση όλων των μηνυμάτων στο νήμα, παρουσιάζοντας τόσο την ερώτηση του χρήστη όσο και την απάντηση του εκπροσώπου. Τα μηνύματα εμφανίζονται με χρονολογική σειρά, περιλαμβάνοντας χρονικές σημάνσεις και δείκτες ρόλων.


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε προσπάθειες για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτοματοποιημένες μεταφράσεις ενδέχεται να περιέχουν λάθη ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
